In [1]:
from bs4 import BeautifulSoup
import json
import requests
import lxml.html as lh
import pandas as pd
import re

/Users/alec/.pyenv/versions/3.6.2/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
def clean(txt):
    return re.sub(
        r"[\s\r\n]",
        "",
        txt
    )

In [3]:
NBSP = u'\xa0'
BOROS = [
    'New York',
    'Bronx',
    'Kings',
    'Queens',
    'Richmond'
]

In [4]:
def get_url_boro(boro_num):
    return 'https://web.enrboenyc.us/CD22509AD'+str(boro_num)+'.html'

In [5]:
def get_url_ad_boro(ad_num, boro_num):
    return 'https://web.enrboenyc.us/CD22509AD'+str(ad_num)+str(boro_num)+'.html'

In [6]:
def get_table(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    return doc.xpath('//table')[2]

In [7]:
def get_ads_in_boro(table):
    ads_in_boro = []
    for tr in table:
        txt = clean(tr[0].text_content())
        if txt != NBSP:
            ads_in_boro.append(txt)
    return ads_in_boro[2:-1]

In [8]:
def get_all_ads_in_boros():
    city = []
    for (i,v) in enumerate(BOROS):
        table = get_table(get_url_boro(i+1))
        ads_in_boro = get_ads_in_boro(table)
        city.append(ads_in_boro)
    return city

In [9]:
def get_eds_in_ad(table, ad_name, boro_name):
    cands = []
    ad = []
    
    for td in table[0][2:-1]:
        cand = td.text_content()
        if cand != NBSP:
            cands.append(cand)


    for tr in table[2:-1]:
        ed_row = []
        for td in tr:
            txt = clean(td.text_content())
            if txt != NBSP and txt != '':
                ed_row.append(txt)
        ed = ed_row[0] ## get ED number
        for (i, result) in enumerate(ed_row[2:]): ## skip % reported column
            ad.append([boro_name, ad_name, ed, cands[i], int(result.replace('-','0'))])
            
    return ad

In [12]:
def get_all_eds_in_ads_in_boros():
    all_eds = []
    city = get_all_ads_in_boros()
    for (i, ads_in_boro) in enumerate(city):
        print('starting boro: ', i+1)
        for ad in ads_in_boro:
            print('scraping ad:', ad)
            table = get_table(get_url_ad_boro(ad[2:4], i+1))
            eds_in_ad = get_eds_in_ad(table, ad, BOROS[i])
            all_eds = all_eds + eds_in_ad
    return all_eds

In [14]:
raw_data = get_all_eds_in_ads_in_boros()

starting boro:  1
scraping ad: AD65
scraping ad: AD66
scraping ad: AD67
scraping ad: AD68
scraping ad: AD69
scraping ad: AD70
scraping ad: AD71
scraping ad: AD72
scraping ad: AD73
scraping ad: AD74
scraping ad: AD75
scraping ad: AD76
starting boro:  2
scraping ad: AD77
scraping ad: AD78
scraping ad: AD79
scraping ad: AD80
scraping ad: AD81
scraping ad: AD82
scraping ad: AD83
scraping ad: AD84
scraping ad: AD85
scraping ad: AD86
scraping ad: AD87
starting boro:  3
scraping ad: AD41
scraping ad: AD42
scraping ad: AD43
scraping ad: AD44
scraping ad: AD45
scraping ad: AD46
scraping ad: AD47
scraping ad: AD48
scraping ad: AD49
scraping ad: AD50
scraping ad: AD51
scraping ad: AD52
scraping ad: AD53
scraping ad: AD54
scraping ad: AD55
scraping ad: AD56
scraping ad: AD57
scraping ad: AD58
scraping ad: AD59
scraping ad: AD60
scraping ad: AD64
starting boro:  4
scraping ad: AD23
scraping ad: AD24
scraping ad: AD25
scraping ad: AD26
scraping ad: AD27
scraping ad: AD28
scraping ad: AD29
scraping a

In [15]:
data = pd.DataFrame(raw_data, columns=['County','AD','ED','Name','Votes'])
data.to_csv('./data/data.csv', index=False)